# Facial expression classification based on VGG16 model

In [ ]:
!nvidia-smi

## i. Import packages

In [ ]:
import numpy as np
import pandas as pd
import os
import csv

from sklearn.model_selection import train_test_split
from tensorflow.keras.activations import swish

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.metrics import categorical_accuracy
from keras.callbacks import ModelCheckpoint
from keras.optimizers import *
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras import regularizers

## ii. Mount the Google Drive to Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/ML_project/data/

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1c2j1ue8z_O6YR92z3E0-7_9xM6v2kjXx/ML_project/data


## iii. Create the folders named "model" and "result"

In [ ]:
folder_model = '/content/drive/MyDrive/ML_project/model'
if not os.path.exists(folder_model):
    os.makedirs(folder_model)

folder_result = '/content/drive/MyDrive/ML_project/result'
if not os.path.exists(folder_result):
    os.makedirs(folder_result)

## iv. Data pre-processing

In [ ]:
def getData(filname):
    Y = []
    X = []
    first = True
    for line in open(filname):
        if first:
            first = False
        else:
            row = line.split(',')
            Y.append(int(row[0]))
            X.append([int(p) for p in row[1].split()])
    X, Y = np.array(X) / 255.0, np.array(Y)
    return X, Y

# Training data
filname = '/content/drive/MyDrive/ML_project/data/train.csv'
X, Y = getData(filname)
num_class = len(set(Y))
N, D = X.shape
X = X.reshape(N, 48, 48, 1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)    # split the training data
y_train = (np.arange(num_class) == y_train[:, None]).astype(np.float32)
y_test = (np.arange(num_class) == y_test[:, None]).astype(np.float32)

# Test data
fileTest = '/content/drive/MyDrive/ML_project/data/test.csv'
X_, Y_ = getData(fileTest)
N_, D_ = X_.shape
X_ = X_.reshape(N_, 48, 48, 1)

# Data augmentation
datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
datagen.fit(X_train)

## v. Model architecture based on VGG16

In [ ]:
def cnn_model():
    model = Sequential()

    # Block 1
    model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation=swish, input_shape=(48, 48, 1)))
    model.add(BatchNormalization())
    model.add(Conv2D(64,(3,3), padding='same', activation=swish))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))

    # Block 2
    model.add(Conv2D(128,(3,3), padding='same', activation=swish))
    model.add(BatchNormalization())
    model.add(Conv2D(128,(3,3), padding='same', activation=swish))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    
    # Block 3
    model.add(Conv2D(256,(3,3), padding='same', activation=swish))
    model.add(BatchNormalization())
    model.add(Conv2D(256,(3,3), padding='same', activation=swish))
    model.add(BatchNormalization())
    model.add(Conv2D(256,(3,3), padding='same', activation=swish))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))

    # Block 4
    model.add(Conv2D(512,(3,3), padding='same', activation=swish))
    model.add(BatchNormalization())
    model.add(Conv2D(512,(3,3), padding='same', activation=swish))
    model.add(BatchNormalization())
    model.add(Conv2D(512,(3,3), padding='same', activation=swish))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))

    # Block 5
    model.add(Conv2D(512,(3,3), padding='same', activation=swish))
    model.add(BatchNormalization())
    model.add(Conv2D(512,(3,3), padding='same', activation=swish))
    model.add(BatchNormalization())
    model.add(Conv2D(512,(3,3), padding='same', activation=swish))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))

    # FC layers
    model.add(Flatten())
    model.add(Dense(256, activation = swish))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(512, activation = swish))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(7, activation='softmax', kernel_regularizer=regularizers.l2(1e-4)))

    # config the model with losses and metrics
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    
    return model

# Show the architecture
model = cnn_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 128)       7

## vi. Train the model and save it as model_team8.h5 file

In [ ]:
%cd /content/drive/MyDrive/ML_project/model/
path_model='model_team8.h5'    # save model at this location after each epoch
K.clear_session()    # destroys the current graph and builds a new one
model=cnn_model()    # create the model
K.set_value(model.optimizer.lr,1e-3)    # set the learning rate
h = model.fit_generator(datagen.flow(X_train, y_train, batch_size=64), 
                    steps_per_epoch=X_train.shape[0]/64, 
                    epochs=320,
                    validation_data=(X_test,y_test),
                    validation_steps=X_test.shape[0]/64,
                    shuffle=True,
                    callbacks=[
                        ModelCheckpoint(filepath=path_model),
                    ]
                    )

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/320
406/406 [==============================] - 85s 86ms/step - loss: 2.2825 - accuracy: 0.1869 - val_loss: 1.9233 - val_accuracy: 0.1582
Epoch 2/320
406/406 [==============================] - 33s 81ms/step - loss: 1.9114 - accuracy: 0.2138 - val_loss: 1.8315 - val_accuracy: 0.2122
Epoch 3/320
406/406 [==============================] - 33s 82ms/step - loss: 1.8528 - accuracy: 0.2395 - val_loss: 1.8682 - val_accuracy: 0.1648
Epoch 4/320
406/406 [==============================] - 34s 83ms/step - loss: 1.8158 - accuracy: 0.2572 - val_loss: 3.7821 - val_accuracy: 0.1720
Epoch 5/320
406/406 [==============================] - 34s 83ms/step - loss: 1.7179 - accuracy: 0.3167 - val_loss: 1.5337 - val_accuracy: 0.4071
Epoch 6/320
406/406 [==============================] - 34s 83ms/step - loss: 1.5226 - accuracy: 0.4109 - val_loss: 1.4478 - val_accuracy: 0.4451
Epoch 7/320
406/406 [==============================] - 34s 84ms/step - loss: 1.4219 - accuracy: 0.4550 - val_loss: 1.3373 - val_ac

## vii. Make predictions and export as prediction_team8.csv file

In [ ]:
y_pred = model.predict(X_)
y_pred = np.argmax(y_pred, axis=1)
header = ['Id', 'Category']
f = open('/content/drive/MyDrive/ML_project/result/prediction_team8.csv', 'w', encoding='UTF8', newline='')
writer = csv.writer(f)
writer.writerow(header)
for i in range(y_pred.shape[0]):
    row = [i, y_pred[i]]
    writer.writerow(row)
f.close()